In [ ]:
import os
from ase.db import connect
from urllib.request import urlretrieve
from mpcontribs.client import Client
from tqdm.notebook import tqdm

In [ ]:
name = 'dtu'
client = Client()
db = 'https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db'

**Retrieve and update project info**

In [ ]:
# client.projects.update_entry(pk=name, project={
#     'references': [
#         {"label": "AEM", "url": "https://doi.org/10.1002/aenm.201400915"},
#         {"label": "PRA", "url": "https://doi.org/10.1103/PhysRevA.51.1944"},
#         {"label": "PRB", "url": "https://doi.org/10.1103/PhysRevB.82.115106"},
#         {"label": "Database", "url": db}
#     ]
# }).result()
client.get_project(name).pretty()

**Create contributions**

In [ ]:
dbdir = "/Users/patrick/gitrepos/mp/MPContribs/mpcontribs-data"
dbfile = db.rsplit('/', 1)[-1]
dbpath = os.path.join(dbdir, dbfile)
if not os.path.exists(dbpath):
    urlretrieve(db, dbpath) 

con = connect(dbpath)
nr_mpids = con.count(selection='mpid')
print(nr_mpids)

In [ ]:
contributions = []

with tqdm(total=nr_mpids) as pbar:
    for row in con.select('mpid'):
        contributions.append({
            'project': name, 'identifier': f'mp-{row.mpid}', 'is_public': True,
            'data': {
                'ΔE': {
                    'KS': { # kohn-sham band gap
                        'indirect': f'{row.gllbsc_ind_gap - row.gllbsc_disc} eV',
                        'direct': f'{row.gllbsc_dir_gap - row.gllbsc_disc} eV'            
                    },
                    'QP': { # quasi particle band gap
                        'indirect': f'{row.gllbsc_ind_gap} eV',
                        'direct': f'{row.gllbsc_dir_gap} eV'           
                    }
                },
                'C': f'{row.gllbsc_disc} eV' # derivative discontinuity
            }
        })
        pbar.update(1)

**Submit contributions**

In [ ]:
# client.delete_contributions(name)
client.submit_contributions(contributions)

**Query contributions**

In [ ]:
query = {
    "project": name,
    "formula__contains": "Mg",
    "data__C__value__gte": 2500,
    "data__ΔE__QP__direct__value__gte": 8000,
    "_order_by": "data__C__value",
    "order": "desc",
    "_fields": [
        "id", "identifier", "formula",
        "data.C.value", "data.ΔE.QP.direct.value"
    ]
}
client.contributions.get_entries(**query).result()